In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from warnings import filterwarnings
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from PIL import Image
from nltk.corpus import stopwords
from nltk.sentiment import SentimentIntensityAnalyzer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate
from sklearn.preprocessing import LabelEncoder
from textblob import Word, TextBlob
from wordcloud import WordCloud
import re

In [ ]:
filterwarnings('ignore')
pd.set_option('display.max_columns', None)
pd.set_option('display.width', 200)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# **Text Preprocessing**

In [ ]:
df=df = pd.read_csv("/kaggle/input/turkcehaber/data.csv")
df.head()

In [ ]:
dff=df.copy()

In [ ]:
dff.rename(columns={'Haber Gövdesi': 'haber', 'Sınıf': 'sinif'}, inplace=True)

In [ ]:
dff.head()

In [ ]:
dff.describe(include='all')

In [ ]:
dff.haber.duplicated(keep="first").value_counts()

In [ ]:
dff.drop_duplicates(subset="haber",keep="first",inplace=True,ignore_index=True)

In [ ]:
dff.describe()

In [ ]:
dff.sinif.unique()

In [ ]:
dff["sinif"].value_counts()

# **Analyze Texts**

In [ ]:
dff['words'] = [len(x.split()) for x in dff['haber'].tolist()]

In [ ]:
dff

In [ ]:
dff['words'].describe()

In [ ]:
dff = dff[~(dff["words"] == 0)]

In [ ]:
dff['words'].describe()

In [ ]:
dff.groupby(['sinif'])['words'].describe()

In [ ]:
min_word_size = 100
max_word_size = 1500
print(dff["sinif"][dff['words'] < min_word_size].value_counts())
print(dff["sinif"][dff['words'] > max_word_size].value_counts())

In [ ]:
dff= dff[dff['words']>=min_word_size]
dff= dff[dff['words']<=max_word_size]

In [ ]:
dff.groupby(['sinif'])['words'].describe()

In [ ]:
dff["sinif"].value_counts()

In [ ]:
df_new= dff[(dff['sinif'] == "Siyaset") & (dff['words'] > 150)]

In [ ]:
dff = dff.drop(df_new.index)

In [ ]:
df_new = dff[(dff['sinif'] == "Magazin") & (dff['words'] > 200)]

In [ ]:
dff = dff.drop(df_new.index)

In [ ]:
df_new = dff[(dff['sinif'] == "Spor") & (dff['words'] > 200)]

In [ ]:
dff = dff.drop(df_new.index)

In [ ]:
df_new = dff[(dff['sinif'] == "Sağlık") & (dff['words'] > 300)]

In [ ]:
dff = dff.drop(df_new.index)

In [ ]:
df_new=dff[(dff['sinif'] == "Bilim-Teknoloji") & (dff['words'] < 170)]

In [ ]:
dff = dff.drop(df_new.index)

In [ ]:
df_new = dff[(dff['sinif'] == "Kültür-Sanat") & ((dff['words'] < 120) | (dff['words'] > 220))]

In [ ]:
dff = dff.drop(df_new.index)

In [ ]:
dff["sinif"].value_counts()

In [ ]:
dff.groupby(['sinif'])['words'].describe()

In [ ]:
target_classes = ['Siyaset', 'Magazin', 'Spor','Kültür-Sanat','Sağlık','Bilim-Teknoloji','Turizm','Finans-Ekonomi','Çevre']
target_df = dff[dff['sinif'].isin(target_classes)]

# Her bir hedef sınıf için alt örnekleme işlemini gerçekleştirelim
undersampled_df = pd.concat([
    target_df[target_df['sinif'] == 'Siyaset'].sample(n=target_df['sinif'].value_counts().min(), replace=False),
    target_df[target_df['sinif'] == 'Magazin'].sample(n=target_df['sinif'].value_counts().min(), replace=False),
    target_df[target_df['sinif'] == 'Spor'].sample(n=target_df['sinif'].value_counts().min(), replace=False),
    target_df[target_df['sinif'] == 'Kültür-Sanat'].sample(n=target_df['sinif'].value_counts().min(), replace=False),
    target_df[target_df['sinif'] == 'Sağlık'].sample(n=target_df['sinif'].value_counts().min(), replace=False),
    target_df[target_df['sinif'] == 'Kültür-Sanat'].sample(n=target_df['sinif'].value_counts().min(), replace=False),
    target_df[target_df['sinif'] == 'Sağlık'].sample(n=target_df['sinif'].value_counts().min(), replace=False),
    target_df[target_df['sinif'] == 'Bilim-Teknoloji'].sample(n=target_df['sinif'].value_counts().min(), replace=False),
    target_df[target_df['sinif'] == 'Turizm'].sample(n=target_df['sinif'].value_counts().min(), replace=False),
    target_df[target_df['sinif'] == 'Finans-Ekonomi'].sample(n=target_df['sinif'].value_counts().min(), replace=False),
    target_df[target_df['sinif'] == 'Çevre'].sample(n=target_df['sinif'].value_counts().min(), replace=False),
    dff[~dff['sinif'].isin(target_classes)]  # Diğer sınıfları olduğu gibi alalım
])

print("\nAlt Örnekleme Sonrası Sınıf Dağılımları:")
print(undersampled_df['sinif'].value_counts())

In [ ]:
undersampled_df.groupby(['sinif'])['words'].describe()

In [ ]:
undersampled_df

In [ ]:
import string
def clean_text(text):
    text = text.replace('İ', 'i')
    text = text.replace('ı', 'i')
    text = text.replace('İ', 'i')
    text = text.replace('Ç', 'c')
    text = text.replace('ç', 'c')
    text = text.replace('Ğ', 'g')
    text = text.replace('ğ', 'g')
    text = text.replace('Ö', 'o')
    text = text.replace('ö', 'o')
    text = text.replace('Ş', 's')
    text = text.replace('ş', 's')
    text = text.replace('Ü', 'u')
    text = text.replace('ü', 'u')
    text = text.lower()
    text = re.sub(r'\d+•', '', text)
    text = re.sub(r'<[^>]+>', '', text)
    text = text.translate(str.maketrans("", "", string.punctuation))
    text = re.sub(r'\s+', ' ', text).strip()

    return text

In [ ]:
undersampled_df["haber"] = undersampled_df["haber"].apply(clean_text)

In [ ]:
undersampled_df

In [ ]:
word_freq= undersampled_df.haber.str.split(expand=True).stack().value_counts()


In [ ]:
word_freq=word_freq.reset_index(name='freq').rename(columns={'index': 'word'})

In [ ]:
top_50_frequent_words = word_freq[:50]
top_50_frequent_words

In [ ]:
tr_stop_words = pd.read_csv('/kaggle/input/trstopwordsss/tr_stop_word.txt',header=None)
print("First 5 entries:")
for each in tr_stop_words.values[:5]:
  print(each[0])

In [ ]:
for each in top_50_frequent_words['word']:
  if each in tr_stop_words.values:
    print (each)

In [ ]:
def remove_stopwords(text):
    words = text.split()
    clean_words = [word for word in words if word.lower() not in tr_stop_words.values]
    return ' '.join(clean_words)

In [ ]:
undersampled_df["haber"] =undersampled_df["haber"].apply(remove_stopwords)

In [ ]:
undersampled_df = undersampled_df.set_index('words')  


In [ ]:
undersampled_df

In [ ]:
selected_category = "Siyaset"
selected_texts = undersampled_df[undersampled_df['sinif'] == selected_category]['haber']
merged_text = ' '.join(selected_texts)

wordcloud = WordCloud(width=800, height=800, background_color='white', min_font_size=10).generate(merged_text)

plt.figure(figsize=(8, 8), facecolor=None)

plt.imshow(wordcloud)
plt.title('Siyaset')
plt.axis("off")
plt.tight_layout(pad=0)

plt.show()

In [ ]:
selected_category = "Bilim-Teknoloji"
selected_texts = undersampled_df[undersampled_df['sinif'] == selected_category]['haber']
merged_text = ' '.join(selected_texts)

wordcloud = WordCloud(width=800, height=800, background_color='white', min_font_size=10).generate(merged_text)

plt.figure(figsize=(8, 8), facecolor=None)

plt.imshow(wordcloud)
plt.title('Bilim-Teknoloji')
plt.axis("off")
plt.tight_layout(pad=0)

plt.show()

In [ ]:
selected_category = "Kültür-Sanat"
selected_texts = undersampled_df[undersampled_df['sinif'] == selected_category]['haber']
merged_text = ' '.join(selected_texts)

wordcloud = WordCloud(width=800, height=800, background_color='white', min_font_size=10).generate(merged_text)

plt.figure(figsize=(8, 8), facecolor=None)

plt.imshow(wordcloud)
plt.title('Kültür-sanat')
plt.axis("off")
plt.tight_layout(pad=0)

plt.show()

In [ ]:
selected_category = "Çevre"
selected_texts = undersampled_df[undersampled_df['sinif'] == selected_category]['haber']
merged_text = ' '.join(selected_texts)

wordcloud = WordCloud(width=800, height=800, background_color='white', min_font_size=10).generate(merged_text)

plt.figure(figsize=(8, 8), facecolor=None)

plt.imshow(wordcloud)
plt.title('Çevre')
plt.axis("off")
plt.tight_layout(pad=0)

plt.show()

In [ ]:
selected_category = "Magazin"
selected_texts = undersampled_df[undersampled_df['sinif'] == selected_category]['haber']
merged_text = ' '.join(selected_texts)

wordcloud = WordCloud(width=800, height=800, background_color='white', min_font_size=10).generate(merged_text)

plt.figure(figsize=(8, 8), facecolor=None)

plt.imshow(wordcloud)
plt.title('Magazin')
plt.axis("off")
plt.tight_layout(pad=0)

plt.show()

In [ ]:
undersampled_df.to_csv('undersampled_data.csv') 

